# pdf 다루기

## pdf 문서에서 텍스트 추출하기

파이썬 라이브러리 <a href="https://pypi.org/project/tika/">tika</a>를 통해 간단하게 텍스트를 추출해서 메모장에 저장하는 코드를 작성해보자. (여러 라이브러리가 있지만 그 중에서 tika가 비교적 성능이 좋고 사용법이 간편하다.)

<pre>pip install tika</pre>

In [ ]:
from tika import parser

data = parser.from_file("files/예시.pdf")
content = data["content"].strip()

txt = open("output.txt", 'w', encoding='utf-8')
print(content, file=txt)
txt.close()

pdf 파일에서 텍스트를 추출할 때는 줄바꿈(개행) 이슈가 존재한다.

## pdf 파일에서 특정 페이지 추출/분할하기

<a href="https://pypi.org/project/PyPDF2/">PyPDF2</a>라는 라이브러리를 사용하면 쉽게 페이지를 추출/분할할 수 있다.

<pre>pip install PyPDF2</pre>

In [ ]:
from PyPDF2 import PdfFileWriter, PdfFileReader

원본 = PdfFileReader(open("files/예시.pdf", 'rb'))

# writer라는 이름의 pdf작성기를 준비한다.
writer = PdfFileWriter()

# 원본 pdf파일에서 원하는 페이지를 추출해서 writer에 추가한다.
writer.addPage(원본.getPage(0))
writer.addPage(원본.getPage(1))
writer.addPage(원본.getPage(4))

# writer가 새 pdf파일로 저장한다.
writer.write(open("files/output.pdf", 'wb'))

반복문을 활용하면 일일이 페이지 번호를 지정하지 않고, 특정 페이지 시작과 끝을 지정해서 작성할 수 있다.

In [ ]:
from PyPDF2 import PdfFileWriter, PdfFileReader

원본 = PdfFileReader(open("files/예시.pdf", 'rb'))

writer = PdfFileWriter()

for 페이지 in range(1, 10):
    writer.addPage(원본.getPage(페이지))
    
writer.write(open("files/output.pdf", 'wb'))

## 여러개의 pdf 파일 병합하기

pdf 병합도 PyPDF2 를 사용한다.

In [ ]:
from PyPDF2 import PdfFileMerger, PdfFileReader

# merger라는 이름의 pdf병합기를 준비한다.
merger = PdfFileMerger()

# 준비한 파일들을 하나씩 읽은 후, merger에 추가한다.
merger.append(PdfFileReader(open("files/1.pdf", 'rb')))
merger.append(PdfFileReader(open("files/2.pdf", 'rb')))
merger.append(PdfFileReader(open("files/3.pdf", 'rb')))
merger.append(PdfFileReader(open("files/4.pdf", 'rb')))
merger.append(PdfFileReader(open("files/5.pdf", 'rb')))

# merger가 새 pdf파일로 저장한다.
merger.write("files/output.pdf")

반복문을 활용하면 일일이 파일명을 지정하지 않고, 하나로 쉽게 합칠 수 있다.

In [ ]:
from PyPDF2 import PdfFileMerger, PdfFileReader
from pathlib import Path

merger = PdfFileMerger()

for 파일 in Path("files").iterdir():
    merger.append(PdfFileReader(open(파일, 'rb')))
    
merger.write("files/output.pdf")

## MS오피스 문서 pdf 파일로 저장하기

<a href="https://github.com/mhammond/pywin32">win32com</a>이라는 라이브러리는 윈도우에서 동작하는 MS오피스 프로그램을 직접 다룰 수 있게 도와준다.

<pre> pip install pywin32 </pre>

그런데 이 라이브러리를 사용할 때는 파일의 "절대경로"를 "문자열" 타입으로 사용해야 한다는 점에 주의해야 한다. <code>pathlib.Path(파일명).resolve()</code>를 사용해 절대경로를 구한 후, 이를 <code>str()</code>함수를 사용해 문자열로 변환시키면 된다.

In [ ]:
from pathlib import Path

파일경로 = str(Path("files/test.pptx").resolve())
저장경로 = str(Path("files/test.pdf").resolve())

print(파일경로)
print(저장경로)

### Word

저장할 때 <code>FileName=저장경로</code>에 이어서 <code>FileFormat=17</code>을 명시하는 점에 주목하자.

In [ ]:
from win32com.client import Dispatch
from pathlib import Path

파일경로 = str(Path("files/test.docx").resolve())
저장경로 = str(Path("files/워드.pdf").resolve())

word = Dispatch("Word.Application")
word.Visible = True

file = word.Documents.Open(파일경로)
file.SaveAs(FileName=저장경로, FileFormat=17)

file.Close()
file = None

word.Quit()

### Powerpoint

저장할 때 <code>FileName=저장경로</code>에 이어서 <code>FileFormat=32</code>을 명시하는 점에 주목하자.

In [ ]:
from win32com.client import Dispatch
from pathlib import Path

파일경로 = str(Path("files/test.pptx").resolve())
저장경로 = str(Path("files/피피티.pdf").resolve())

powerpoint = Dispatch("Powerpoint.Application")
powerpoint.Visible = True

file = powerpoint.Presentations.Open(파일경로)
file.SaveAs(FileName=저장경로, FileFormat=32)

file.Close()
file = None

powerpoint.Quit()

### Excel

저장할 때 <code>Type=0</code>을 먼저 명시한 후, 이어서 <code>Filename=저장경로</code>를 적어주는 점에 주목하자.

In [ ]:
from win32com.client import Dispatch
from pathlib import Path

파일경로 = str(Path("files/test.xlsx").resolve())
저장경로 = str(Path("files/엑셀.pdf").resolve())

excel = Dispatch("Excel.Application")
excel.Visible = True

file = excel.Workbooks.Open(파일경로)
file.ExportAsFixedFormat(Type=0, Filename=저장경로)

file.Close()
file = None

excel.Quit()

# (실습) 엑셀을 활용해 대량의 개인별 pdf 보고서 만들기

In [ ]:
from win32com.client import Dispatch
from pathlib import Path

파일경로 = str(Path("files/보고서.xlsx").resolve())

excel = Dispatch("Excel.Application")
excel.Visible = True

file = excel.Workbooks.Open(파일경로)

# 시트 이름 지정
report = file.Worksheets("보고서")
# 시트 선택
report.Select()

for 번호 in range(1,6):
    # 특정 셀에 번호 입력
    report.Cells(2, 1).Value = 번호
    # 해당 이름으로 보고서 저장    
    이름 = report.Cells(2, 2).Value
    저장경로 = str(Path(f"{번호}_{이름}.pdf").resolve())
    # 현재 활성화된 시트(ActiveSheet)를 저장
    file.ActiveSheet.ExportAsFixedFormat(Type=0, Filename=저장경로)

# 변경사항 저장 False
file.Close(False)
file = None

excel.Quit()